In [1]:
%pip install --no-deps duckdb
%pip install --no-deps chromadb python-dotenv
%pip install -q pyarrow pandas tqdm langchain-community

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, re, math, json, time
from pathlib import Path
import pandas as pd
import duckdb
from tqdm.auto import tqdm
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import numpy as np, math

In [3]:
# Config & paths
DATA_DIR   = Path("data/processed")   # Parquet input
CHROMA_DIR = Path("vector_store")     # will be created
DUCK_FILE  = "meta.duckdb"            # output

EMBED_MODEL = "nomic-embed-text"      # must exist in `ollama list`

CHROMA_DIR.mkdir(exist_ok=True, parents=True)

In [4]:
df = pd.read_parquet(DATA_DIR / "movies.parquet")
print("Rows loaded:", len(df))
df.head(2)

Rows loaded: 19986


,movie_id,title,tagline,overview,vote_average,runtime,release_date,genres,keywords_list,providers_list
0,1363475,Into the Reds Tale,,An abomination mimic creature enters the story...,0.0,4,2025-06-27,"[Animation, Horror, Fantasy]",[],[]
1,1355895,Pader,,"A young boy, clad in a dunce cap and lion's pa...",0.0,8,2024-09-01,[],[],[]


In [5]:
# Minimal clean & augment for any stray html tags
def strip_html(txt: str) -> str:
    if not isinstance(txt, str): 
        return ""
    return re.sub("<[^>]+>", " ", txt).replace("\n", " ").strip()
    
def to_list_of_str(obj):
    "Always return a plain Python list[str] (no ndarray)."
    if obj is None or (isinstance(obj, float) and math.isnan(obj)):
        return []
    if isinstance(obj, np.ndarray):
        return [str(x) for x in obj.tolist()]
    if isinstance(obj, (list, tuple, set)):
        return [str(x) for x in obj]
    return [str(obj)]

df["title"]    = df["title"].fillna("").str.strip()
df["tagline"]  = df["tagline"].fillna("").apply(strip_html).str.lower()
df["overview"] = df["overview"].fillna("").apply(strip_html).str.lower()

# derive numeric columns
df["release_year"] = pd.to_datetime(df["release_date"], errors="coerce").dt.year
df["decade"]       = (df["release_year"] // 10) * 10

# primitive list creation, because it causes problems with embeddings
df["genres"]         = df["genres"].apply(to_list_of_str)
df["providers_list"] = df["providers_list"].apply(to_list_of_str)
df["keywords_list"]  = df["keywords_list"].apply(to_list_of_str)

# sanity check
assert df["genres"].apply(lambda g: isinstance(g, list)).all()

In [6]:
# Build `text_for_embed`
def make_text(row):
    kw_list = row["keywords_list"]
    if isinstance(kw_list, float) and math.isnan(kw_list):
        kw_list = []                      # handle NaN
    kw_list = list(kw_list)               # NumPy → Python list
    kws = ", ".join(kw_list) if len(kw_list) else ""
    
    parts = [row["title"], row["tagline"], row["overview"], f"keywords: {kws}"]
    return ". ".join(p for p in parts if p)

df["text_for_embed"] = df.apply(make_text, axis=1)
df["text_for_embed"].str.slice(0, 120).head()


0    Into the Reds Tale. an abomination mimic creat...
1    Pader. a young boy, clad in a dunce cap and li...
2    Dadfest. the long-simmering tension between tw...
3    The Deadline. you can't escape it!. a writer s...
4    Pollastra All Along. keywords: psicologia, hil...
Name: text_for_embed, dtype: object

In [7]:
# Instantiate embeddings  (Ollama must be running)
embed_fn = OllamaEmbeddings(model=EMBED_MODEL)

/var/folders/gq/7yf_r66s183874jprrn7q7440000gn/T/ipykernel_82285/215189986.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embed_fn = OllamaEmbeddings(model=EMBED_MODEL)


In [11]:
# Create / populate Chroma collection  (~25–35 min for 200 k rows on M-series CPU)
# needed to account for chroma db list compatibility
def stringify_lists(meta: dict) -> dict:
    """Turn any list into a |-separated string; leave primitives unchanged."""
    out = {}
    for k, v in meta.items():
        if isinstance(v, list):
            out[k] = "|".join(v)        # "Animation|Horror|Fantasy"
        else:
            out[k] = v
    return out

movies_vs = Chroma(
    collection_name      = "movies",
    embedding_function   = embed_fn,
    persist_directory    = str(CHROMA_DIR),
)

docs      = df["text_for_embed"].tolist()
metadatas = df.apply(
    lambda r: {
        "movie_id":  int(r.movie_id),
        "title":     r.title,
        "genres":    r.genres,          # now plain list[str]
        "providers": r.providers_list,  # plain list[str]
        "vote_average": float(r.vote_average) if not math.isnan(r.vote_average) else None,
        "decade":       int(r.decade) if not math.isnan(r.decade) else None,
    },
    axis=1
).tolist()
ids = df.movie_id.astype(str).tolist()  

metadatas_safe = [stringify_lists(m) for m in metadatas]

t0 = time.time()

BATCH = 5_000                    # keep well under 5 461
total = len(docs)

for i in tqdm(range(0, total, BATCH), desc="chroma upsert"):
    j = i + BATCH
    movies_vs.add_texts(
        texts     = docs[i:j],
        metadatas = metadatas_safe[i:j],   # use the list-stringified version from earlier
        ids       = ids[i:j],
    )

movies_vs.persist()              # one flush at the end
print("Ingest finished")

print(f"Chroma ingest done in {time.time()-t0:,.0f} s")


chroma upsert:   0%|          | 0/4 [00:00<?, ?it/s]

Ingest finished
Chroma ingest done in 337 s


/var/folders/gq/7yf_r66s183874jprrn7q7440000gn/T/ipykernel_82285/1172071956.py:48: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  movies_vs.persist()              # one flush at the end


In [12]:
# Write DuckDB metadata side-store
meta_cols = ["movie_id", "title", "vote_average", "decade", "runtime", "providers_list", "genres"]
con = duckdb.connect(DUCK_FILE)
con.register("movies_df", df[meta_cols])
con.execute("CREATE OR REPLACE TABLE movies_meta AS SELECT * FROM movies_df")
con.close()
print("DuckDB rows:", len(df))

DuckDB rows: 19986


In [30]:
# Smoke-test similarity search
vs = Chroma(
    collection_name   = "movies",
    embedding_function= embed_fn,
    persist_directory = str(CHROMA_DIR),
)
hits = vs.similarity_search(
    query  = "eccentric history film",
    k      = 100
)
# filter = {"providers": {"$in": ["Disney+"]}}
for h in hits:
    print(f"{h.metadata['title']}  — vote_avg: {h.metadata['vote_average']} providers: {h.metadata['providers'][:3]}")


Time Travelers Overture  — vote_avg: 0.0 providers: 
The Parallel Coast  — vote_avg: 0.0 providers: 
Half Way Back  — vote_avg: 0.0 providers: 
Make Good Choices  — vote_avg: 0.0 providers: 
Nine Awkward Dates  — vote_avg: 0.0 providers: 
Holmie Day  — vote_avg: 0.0 providers: 
Scraggliness  — vote_avg: 0.0 providers: 
it's called round like a head  — vote_avg: 0.0 providers: 
A Thousand Untaken Paths  — vote_avg: 6.0 providers: 
Any Place But Here  — vote_avg: 0.0 providers: 
Our Time Together  — vote_avg: 0.0 providers: 
A Stone Story  — vote_avg: 0.0 providers: 
La Fin des slows  — vote_avg: 5.0 providers: 
Coin Toss  — vote_avg: 0.0 providers: 
Pointing to the Sun and Laughing (On a Stack of Poems)  — vote_avg: 0.0 providers: 
time has no place here  — vote_avg: 0.0 providers: 
Time Metallurgist  — vote_avg: 0.0 providers: 
Delirios  — vote_avg: 0.0 providers: 
Hysterical  — vote_avg: 0.0 providers: 
The Cult of the Parsnip  — vote_avg: 0.0 providers: 
Man with a Movie Camera  — vo